In [2]:
import pandas as pd
import numpy as np
import re, time, requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)

import pyautogui
import pyperclip
import itertools
#from fake_useragent import UserAgent

In [ ]:
#pip install selenium

In [3]:
#Step 1: use Bardeen to get IDs from Facebook search result
#Step 2: get poster's ID
ID_df = pd.read_excel("保養品ID.xlsx") #This is where you read the ID file, you can change the name to yours. 
ID_df["ID"] = ID_df["Link"].str.split("/").str[3]
ID_df["ID"] = ID_df["ID"].str.split("?").str[0]
ID_df = ID_df[ID_df.ID !="profile.php"]
ID_df = ID_df.drop_duplicates(subset=['ID'], keep='first')
ID_df = ID_df.reset_index(drop=True)

ID_df

,Name,Link,ID
0,HOOK,https://www.facebook.com/helloiamhook/?__cft__[0]=AZWVAM_dK1Zc3MjaPoQHYqVqYOMqRrLYUL7S8A7ENPoZPV3BfJR-sXC47wQzilJdQz_2qipomUPcWm8TZ8BtYdKc_aeT0v-QaTKlvf-6THZ1WKr6Eda0YuDDgFr7iKJfDaw&__tn__=-UC%2CP-R,helloiamhook
1,EDN茵蝶保養品,https://www.facebook.com/EDNcareTW/?__cft__[0]=AZX9QqZCS2ytgfArKOFL3AKelBIET_n4nmTPs8pbrnsf9XE8psrby7uuMdPAmKCIERO2kkF2-0I4uLkNaZeuF0KDmo7gEHprBR50TgtcX28CDBCKG9hhxot1e1NDuCAWC8s&__tn__=-UC%2CP-R,EDNcareTW
2,簡曉育 Vicni,https://www.facebook.com/vicni429/?__cft__[0]=AZVraXyDj0boaft4DR_-IifvhvF-2WV4bfMPiYQhnm-UuG9WC7mUljy88RnjFNz1Thl70_6eQBXAixKbEXdtEQiZTYphpwCgMWKOMstaTkmy9AnIV2EBnwMQ_HWRjZBq5vo&__tn__=-UC%2CP-R,vicni429
3,李佩甄,https://www.facebook.com/janeleetaiwan?__cft__[0]=AZWLbGoiyohYx03nQIe5f74WBk7FJVCAYPj9g3-0BjYgq8tfQWstv5u0gmpqyVgvLE2fXTQB95z8e6zbRlwdBba5F6Ta2wXQrSsXXKbHWeaZJEQLwflhr7S7zZWA2CHCy_XLL3C9oa07ao27QGlzfvzz&__tn__=-UC%2CP-R,janeleetaiwan
4,黃幼鳴皮膚科醫師,https://www.facebook.com/dryuminghuang/?__cft__[0]=AZUHmeDIUwnc1HFq8eQlt3HMsFQu7IrRFFk8SF3vba-X54dYWBxI1NMVjSc-1Ui0Xx_GDnwD02cwRXa1FKAMijP-t6Zh5W4y2NU4LcqX_A2ZX8q_Nf4eo8FCiXexHhL8RsE&__tn__=-UC%2CP-R,dryuminghuang
5,丹妮婊姐星球,https://www.facebook.com/danny0425?__cft__[0]=AZWvwImbxBHps3a8qZX1QIVR00v3rIzvRG4TvrRkKzHsAwSasYafxDDeWzOFovlZSJ-RbQLgRPsUwF--gX9JVk14DAoQGn1L4eem1EB2cpGPskQ1CSscA_Os44iaXpzBVYY&__tn__=-UC%2CP-R,danny0425
6,陳沂,https://www.facebook.com/chenyi326?__cft__[0]=AZWZdxfsSqlhHkrO-5zKocLlSNaqYkImyHuHRbqpd1fkcVaMprc_NR21__6PQ1Y4bjbpapHcBgotoo_63I2BUtTM8CtG384kOUroiN9a0arn5vWbCU6S1ECsXo7CS5dYGQQ&__tn__=-UC%2CP-R,chenyi326
7,豬V小姐,https://www.facebook.com/vickyhuang720314?__cft__[0]=AZW1JgP__2XqZp2eGI0T78ejFGGQgdmR5av5GaEmrcm99WmFs4BJe6jzPAc1SdIJHLKiMChB7V9541YqFHFQLp3fmONQ9BLpLrXwUakGAcZ2F2P__m_8rl_6uqXo73Xu_kI&__tn__=-UC%2CP-R,vickyhuang720314
8,口罩男,https://www.facebook.com/gn00357648?__cft__[0]=AZWQVtoeFO8ICNhReeu50HEjoLrcsFWK30Ih2Y6AABlN1mAatob4aRnlchT1hX7mWmLJOLpLNmsh2bvx4A8mUBIwdzpnpIBY_vuaIc6ut2Ppa48ueL0l5zlDk6QuAOFOE70&__tn__=-UC%2CP-R,gn00357648
9,蕭淑慎KittyHsiao,https://www.facebook.com/L.hsiao.shu.shen?__cft__[0]=AZVzlTSHoCNN2fYBuXKLNXw7kmruYYcJqArKe8WF6be41JrmoPAZIIi5n5lHQueNuCMbh06XmIjpVGfxbH_qheowFI6ejwDkPFzOO5KH50YCaPBiO-K7AHQfBaQh1736HunfZ0icZLFcSv1HRyXr49d2231i1T4-2ihuaqOVVoXEYw&__tn__=-UC%2CP-R,L.hsiao.shu.shen


In [5]:
#Step 3: automated log-in to facebook and click off notifications.
#Get user agent from here: http://httpbin.org/get.

#ua = UserAgent(browsers=["chrome"]) #fake user_agent
#userAgent = ua.random

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36")
#Change your user-agent above

driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.facebook.com")
time.sleep(2)
driver.find_element("name", "email").send_keys("username") #Please change to your FB username
time.sleep(1)
driver.find_element("name", "pass").send_keys("password") #Please change to your FB password
time.sleep(1)
driver.find_element("name", "login").click()
time.sleep(8)

pyautogui.click(377, 207)
time.sleep(1)
pyautogui.click(1724, 414)
time.sleep(5)

In [64]:
#Step 4: specify the ID you want to scrape and get the url of each posts
anchor_list = []

for ID in ID_df["ID"]:
    driver.get("https://www.facebook.com/" + ID)
    for i in range(4): #You can specify how many scrolls-downs to do, the more you do the more posts will be loaded, now is 5 .
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
    
        #Find every links to posts
        #target all the link elements on the page
        try: 
            anchors = driver.find_elements(By.TAG_NAME, "a")
            anchors = [a.get_attribute("href") for a in anchors]
            #narrow down all links to posts links only
            anchors = [a for a in anchors if str(a).startswith("https://www.facebook.com/" + ID + "/posts")]
            anchor_list.append(anchors)
        except:
            print("Loading failed")
            
    print('Found ' + str(len(anchors)) + ' links to posts')

anchor_list = list(itertools.chain(*anchor_list)) # put saveral list into one list
anchor_list = list(set(anchor_list)) #drop duplicates
print('Totally found ' + str(len(anchor_list)) + ' links to posts')

Found 20 links to posts
Found 5 links to posts
Found 21 links to posts
Found 17 links to posts
Found 26 links to posts
Found 19 links to posts
Found 21 links to posts
Found 19 links to posts
Found 20 links to posts
Found 21 links to posts
Found 20 links to posts
Found 20 links to posts
Found 23 links to posts
Found 8 links to posts
Found 15 links to posts
Found 14 links to posts
Found 12 links to posts
Found 15 links to posts
Found 10 links to posts
Found 19 links to posts
Found 4 links to posts
Found 0 links to posts
Found 22 links to posts
Found 19 links to posts
Found 0 links to posts
Found 8 links to posts
Found 17 links to posts
Found 23 links to posts
Found 17 links to posts
Found 20 links to posts
Found 22 links to posts
Found 6 links to posts
Found 21 links to posts
Found 22 links to posts
Found 25 links to posts
Found 0 links to posts
Found 13 links to posts
Found 28 links to posts
Found 30 links to posts
Found 0 links to posts
Found 24 links to posts
Found 26 links to posts
F

In [65]:
#Step 5: Scrape all the posts and comments which has been loaded and save it as csv file.  
post_list = []
comment_list = []

for link in anchor_list:
    driver.get(link)
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    content = soup.find("div", {"x1iorvi4 x1pi30zi x1swvt13 x1l90r2v"})
    comment = soup.find("div", {"class": "x1jx94hy x12nagc"})
    
    #process the post content
    try:
        text_post = str(content).replace('href=', 'start;"')
        text_post = str(text_post).replace('style="text-align:start">', 'style="text-align: start;">')
        text_post = str(text_post).split('start;"')
        text_post = [i for i in text_post if str(i).startswith((">", '"'))]
        text_post = [i.split("<")[0] for i in text_post]
        text_post = [i.replace('"', '>') for i in text_post]
        text_post = [i.split(">")[1] for i in text_post]
        text_post = "\n".join(str(i) for i in text_post)
    except:
        print("Failed")
    post_list.append(text_post)
    
    #process the comment content
    try:
        text_comment = str(comment).split(' ')
        text_comment = [i for i in text_comment if str(i).startswith("start")]
        text_comment = [i.split("<")[0] for i in text_comment]
        text_comment = [i.split('"')[1] for i in text_comment]
        text_comment = "\n".join(str(i) for i in text_comment)
    except:
        print("Failed")
    comment_list.append(text_comment)
    print("Done")
    
df = pd.DataFrame()
df["ID"] = " "
df["Post_link"] = anchor_list
df["Post"] = post_list
df["Comment"] = comment_list
anchor_name = [i.split("/")[3] for i in anchor_list]
df["ID"] = [i.split("?")[0] for i in anchor_name]

#Step 6:Filter out the posts which doesn't contain https.
df["TRUE"] = df["Post"].str.contains("https")
df = df[df.TRUE == True]
df.drop("TRUE", inplace=True, axis=1)
df = df.sort_values(by=['ID'])
df = df.reset_index(drop=True)

result = pd.merge(ID_df, df, how='right')

result.to_csv("Post_content.csv", index=False) #Save as csv file

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Failed
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Don